#### 1. Preparation

In [ ]:
#
import os
import pathlib

import numpy as np
import pandas as pd

import xarray as xr
import rioxarray as rxr
import rasterio as rio

import geopandas as gpd
from shapely.geometry import LineString, Point, MultiPoint, box

import matplotlib.pyplot as plt
import seaborn as sns

import netCDF4

In [ ]:
# Get new path
# Create a path to the folder
# that contains file "VCSN_waikanae_200501_disaggdaily_500m_nztm_masked_sec.nc"
main_dir = r"S:\Grid_Ori_bigdata"

#### 2. Process data

In [ ]:
# Get data
# Get new x and new y
rainfall_cyp_xr = xr.open_dataset(fr"{main_dir}\VCSN_waikanae_200501_disaggdaily_500m_nztm_masked_sec.nc")

# Change variable name
rainfall_cyp_xr['rainfall_depth'] = rainfall_cyp_xr['depth']
rainfall_cyp_xr = rainfall_cyp_xr.drop('depth')

# Assign CRS
rainfall_cyp_xr.rio.write_crs('epsg:2193', inplace=True)

In [ ]:
# Clip data
geometry_domain_clip = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [1767790 - 1150, 5471400 - 1150],
            [1767790 - 1150, 5475880 + 1150],
            [1774510 + 1150, 5475880 + 1150],
            [1774510 + 1150, 5471400 + 1150],
            [1767790 - 1150, 5471400 - 1150]
        ]]
    }
]

In [ ]:
rainfall_clip_first = rainfall_cyp_xr.rio.clip(geometry_domain_clip)

#### 3. Interpolation

In [ ]:
# Check boundaries
ori_rain_bound = rainfall_clip_first.rio.bounds()
ori_rain_bound

In [ ]:
%%time
# Get new x and new y
new_x = np.arange(ori_rain_bound[0]+2.5, ori_rain_bound[2], step=5)
new_y = np.arange(ori_rain_bound[1]+2.5, ori_rain_bound[3], step=5)

# Get interpolation
rainfall_interpolation = rainfall_cyp_xr.interp(x=new_x, y=new_y, method='nearest')
rainfall_interpolation.rio.write_crs('epsg:2193', inplace=True)
rainfall_interpolation.to_netcdf(fr"{main_dir}\rainfall_interpolation_nearest.nc", format="NETCDF4", engine="netcdf4")